## Vectorizing and Clustering

imports

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os
import json
import hdbscan
# Level up one level to add app the the allowed routes
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from app.utils.base_dir import BASE_DIR


### Vectorizing

open the json and load the text and label

In [ ]:


path = BASE_DIR/'data'/'processed'/'processedData.json' 
with open(path, 'r') as f:
    docs = json.load(f)

label = [list(d.keys())[0] for d in docs]
text = [list(d.values())[0] for d in docs]


check if is the text is none

In [ ]:
for i, doc in enumerate(text):
    if doc is None:
        print(f"Documento {i} es None")
    elif not isinstance(doc, str):
        print(f"Documento {i} no es str: {type(doc)} -> {repr(doc)}")
    elif not doc.strip():
        print(f"Documento {i} está vacío")


❌ Documento 12 es None
❌ Documento 17 es None
❌ Documento 21 es None
❌ Documento 35 es None
❌ Documento 36 es None
❌ Documento 42 es None
❌ Documento 43 es None
❌ Documento 45 es None
❌ Documento 46 es None
❌ Documento 48 es None
❌ Documento 49 es None
❌ Documento 54 es None
❌ Documento 68 es None
❌ Documento 70 es None
❌ Documento 71 es None
❌ Documento 72 es None
❌ Documento 75 es None
❌ Documento 87 es None
❌ Documento 91 es None
❌ Documento 92 es None
❌ Documento 113 es None
❌ Documento 119 es None
❌ Documento 120 es None
❌ Documento 121 es None
❌ Documento 123 es None
❌ Documento 125 es None
❌ Documento 127 es None
❌ Documento 142 es None
❌ Documento 161 es None
❌ Documento 168 es None
❌ Documento 171 es None
❌ Documento 174 es None
❌ Documento 175 es None
❌ Documento 177 es None
❌ Documento 181 es None
❌ Documento 183 es None
❌ Documento 192 es None
❌ Documento 211 es None
❌ Documento 217 es None
❌ Documento 223 es None
❌ Documento 227 es None
❌ Documento 228 es None
❌ Documento 

vectorizing the text

In [43]:

def TF_IDF_vectorizer(text_corpus: list):
    vectorizer = TfidfVectorizer(
        max_features= 10000,  # the max num of words for the total corpus 
        max_df=0.85,    #delete if the word apears on the 85% of docs
        min_df=0.01,    #delete if tye word aperas just on the 1% of docs
    )
    return vectorizer.fit_transform(text_corpus)
TF_vector = TF_IDF_vectorizer(text)
print(TF_vector) 

AttributeError: 'NoneType' object has no attribute 'lower'

### Clustering

In [21]:
def cluster_hdbscan(TF_vector):
    clusterer = hdbscan.HDBSCAN()
    # return a object like [0,1,5,0,1,5,6,9,0] where every number in order is the number of cluster of the document
    return clusterer.fit_predict(TF_vector)
    
print(cluster_hdbscan(TF_vector))


[-1 -1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5  5 -1 -1 -1
 -1 -1 -1  5 -1 -1 -1 -1 -1  5 -1 -1 -1 -1  5 -1  5  1 -1 -1 -1 -1  1 -1
 -1  5  1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1  4 -1 -1  1 -1  5  1 -1 -1 -1  5
 -1 -1  1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1  5 -1 -1  1 -1 -1  4 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1 -1  5 -1  5 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  2 -1 -1 -1  4  4 -1
  1 -1  1 -1  5 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  5  2 -1 -1  1 -1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1
 -1  5  4  4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5 -1 -1 -1  5
 -1 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1 -1  1 -1 -1  5 -1  1  1 -1 -1 -1 -1
 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1  4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5  5 -1  1 -1